In [2]:
%load_ext autoreload
%autoreload 2

In [28]:
import pandas as pd
from glob import glob 
from fastcore.xtras import load_pickle

from gpt3forchem.output import get_regression_metrics
from gpt3forchem.api_wrappers import extract_prediction

from pycm import ConfusionMatrix

## Classification

In [29]:
all_res = glob('results/20221130_lipophilicity/*')

In [30]:
len(all_res)

20

In [31]:
metrics = []
baselines = []

for res in all_res:
    res = load_pickle(res)
    cm = res['cm']
    cm = ConfusionMatrix(list(map(lambda x: str(x).strip(), res['cm'].actual_vector)), list(map(lambda x: str(x).strip(), res['cm'].predict_vector)))
    baseline_cm = res['baseline']['cm']
    baseline_cm = ConfusionMatrix(list(map(lambda x: str(x).strip(), baseline_cm.actual_vector)), list(map(lambda x: str(x).strip(), baseline_cm.predict_vector)))
    metrics.append(
        {
            'train_size': res['train_size'],
            'representation': res['representation'],
            'accuracy': cm.ACC_Macro,
            'f1_macro': cm.F1_Macro,
            'f1_micro': cm.F1_Micro
        })
    baselines.append(
        {
            'train_size': res['train_size'],
            'representation': res['representation'],
            'accuracy': baseline_cm.ACC_Macro,
            'f1_macro': baseline_cm.F1_Macro,
            'f1_micro': baseline_cm.F1_Micro  
        })

In [32]:
metrics = pd.DataFrame(metrics)
baselines = pd.DataFrame(baselines)

In [40]:
metrics.groupby(['representation', 'train_size']).agg(['mean', 'std', 'count'])

accuracy                  f1_macro                  \
                               mean       std count      mean       std count   
representation train_size                                                       
inchi          10          0.750000       NaN     1  0.178333       NaN     1   
               50          0.748000       NaN     1  0.108824       NaN     1   
               500         0.754000       NaN     1  0.213226       NaN     1   
iupac_name     10          0.842000       NaN     1  0.081631       NaN     1   
               50          0.710000       NaN     1  0.086275       NaN     1   
               500         0.740000       NaN     1  0.272902       NaN     1   
selfies        10          0.732000  0.025456     2  0.099103  0.014689     2   
               50          0.746000       NaN     1  0.143286       NaN     1   
               500         0.758000       NaN     1  0.269271       NaN     1   
smiles         10          0.752000  0.013367     4  0.190699  0.060255     4   
               50          0.752667  0.003055     3  0.127131  0.015639     3   
               500         0.756667  0.003055     3  0.277105  0.011910     3   

                           f1_micro                  
                               mean       std count  
representation train_size                            
inchi          10          0.375000       NaN     1  
               50          0.370000       NaN     1  
               500         0.385000       NaN     1  
iupac_name     10          0.210000       NaN     1  
               50          0.275000       NaN     1  
               500         0.350000       NaN     1  
selfies        10          0.330000  0.063640     2  
               50          0.365000       NaN     1  
               500         0.395000       NaN     1  
smiles         10          0.380000  0.033417     4  
               50          0.381667  0.007638     3  
               500         0.391667  0.007638     3

In [41]:
baselines.groupby(['train_size']).agg(['mean', 'std', 'count'])

accuracy             f1_macro            f1_micro           
               mean  std count      mean  std count     mean  std count
train_size                                                             
10            0.750  0.0     8  0.109091  0.0     8    0.375  0.0     8
50            0.744  0.0     6  0.174545  0.0     6    0.360  0.0     6
500           0.778  0.0     6  0.249968  0.0     6    0.445  0.0     6

## Regression

In [35]:
all_res_regression = glob('results/20221129_lipophilicity_regression/*')

In [36]:
metrics_regression = []
baselines_regression = []

for res in all_res_regression:
    res = load_pickle(res)
    metrics_regression.append(
        {
            'train_size': res['train_size'],
            'representation': res['representation'],
            'r2': res['metrics']['r2'],
            'max_error': res['metrics']['max_error'],
            'mean_absolute_error': res['metrics']['mean_absolute_error'],
            'mean_squared_error': res['metrics']['mean_squared_error'],
            'rmse': res['metrics']['rmse'],
        })
    baselines_regression.append(
        {
            'train_size': res['train_size'],
            'representation': res['representation'],
            'r2': res['baseline']['r2'],
            'max_error': res['baseline']['max_error'],
            'mean_absolute_error': res['baseline']['mean_absolute_error'], 
            'mean_squared_error': res['baseline']['mean_squared_error'],
            'rmse': res['baseline']['rmse'],
        })

In [37]:
metrics_regression = pd.DataFrame(metrics_regression)

baselines_regression = pd.DataFrame(baselines_regression)

In [42]:
metrics_regression.groupby(['representation', 'train_size']).agg(['mean', 'std', 'count'])

r2                 max_error                  \
                               mean       std count      mean       std count   
representation train_size                                                       
selfies        10         -0.283004  0.250696     2  3.555000  0.360624     2   
               50         -0.101567       NaN     1  3.810000       NaN     1   
               500         0.014101       NaN     1  3.630000       NaN     1   
smiles         10         -0.399444  0.037617     3  4.470000  0.173205     3   
               50         -0.075293  0.078660     3  3.886667  0.321921     3   
               500         0.028272  0.085881     2  3.485000  0.487904     2   

                          mean_absolute_error                  \
                                         mean       std count   
representation train_size                                       
selfies        10                    1.123650  0.132512     2   
               50                    1.058500       NaN     1   
               500                   0.945650       NaN     1   
smiles         10                    1.155050  0.015769     3   
               50                    1.040917  0.033656     3   
               500                   0.953150  0.026022     2   

                          mean_squared_error                      rmse  \
                                        mean       std count      mean   
representation train_size                                                
selfies        10                   1.977891  0.386476     2  1.402999   
               50                   1.745429       NaN     1  1.321147   
               500                  1.425084       NaN     1  1.193769   
smiles         10                   2.157397  0.057991     3  1.468719   
               50                   1.703799  0.124637     3  1.304702   
               500                  1.404601  0.124138     2  1.184580   

                                           
                                std count  
representation train_size                  
selfies        10          0.137732     2  
               50               NaN     1  
               500              NaN     1  
smiles         10          0.019785     3  
               50          0.048242     3  
               500         0.052398     2

In [43]:
baselines_regression.groupby(['train_size']).agg(['mean', 'std', 'count'])

r2            max_error            mean_absolute_error       \
                mean  std count      mean  std count                mean  std   
train_size                                                                      
10         -0.047865  0.0     5  3.701992  0.0     5            0.986299  0.0   
50          0.136901  0.0     4  2.993319  0.0     4            0.915308  0.0   
500         0.359285  0.0     3  3.108279  0.0     3            0.819692  0.0   

                 mean_squared_error                 rmse             
           count               mean  std count      mean  std count  
train_size                                                           
10             5           1.582862  0.0     5  1.258118  0.0     5  
50             4           1.307537  0.0     4  1.143476  0.0     4  
500            3           1.054598  0.0     3  1.026936  0.0     3